In [253]:
import logging
import random
from typing import Callable
from collections import namedtuple
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [254]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [255]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i*2+1 for i in range(num_rows)]
        self._k = k
    
    def __bool__(self):
        return sum(self.rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [256]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

In [257]:
def cook_status(state: Nim) -> dict:
    cooked = dict()
    #ALL THE POSSIBLE MOVES
    cooked['possible_moves'] = [(r, o) for r,c in enumerate(state.rows) for o in range(1, c+1) if state.k is None or o <= state.k]
    #NUMBER OR ROWS "ACTIVE": a row is active when we have at least one element.
    cooked['activate_rows_number'] = sum(o > 0 for o in state.rows)
    #INTEGER representing the index of row with the minimum number of objects 
    cooked['shortest_row'] = min((x for x in enumerate(state.rows) if x[1] > 0), key = lambda y: y[1])[0]
    #INTEGER representing the index of row with the maximum number of objects 
    cooked['longest_row'] = max((x for x in enumerate(state.rows) if x[1] > 0), key = lambda y: y[1])[0]
    #NIM_SUM
    cooked['nim_sum'] = nim_sum(state)
    
    brute_force = list()
    for m in cooked['possible_moves']:
        tmp = deepcopy(state)
        tmp.nimming(m)
        #we append the tuple (m, nim_sum(tmp)) which represents the move and the possible effect (in terms of nim_sum) on the state
        brute_force.append((m, nim_sum(tmp)))
    cooked['brute_force'] = brute_force

    return cooked

# Possible strategies of the other player

In [258]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [259]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [260]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data['brute_force'] if bf[1] == 0), random.choice(data['brute_force']))[0]

## Possible strategies we can play

In [261]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)
        if random.random() < genome['p']:
            ply = Nimply(data['shortest_row'], random.randint(1, state.rows[data['shortest_row']]))
        else:
            ply = Nimply(data['longest_row'], random.randint(1, state.rows[data['longest_row']]))

        return ply
    return evolvable

In [262]:
#DICTIONARY OF PARAMETERS: {"%_taken_longest", "%_taken_shortest", "binary_chance"} 
def make_strategy_evol(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome['binary_chance']:
            ply = Nimply(data['shortest_row'], random.randint(1, int(state.rows[data['shortest_row']]/100*genome['%_taken_shortest'])))    
        else:
            ply = Nimply(data['longest_row'], random.randint(1, int(state.rows[data['longest_row']]/100*genome['%_taken_longest']))) 
        
        return ply
    return evolvable

In [263]:
NUM_MATCHES = 100
NIM_SIZE = 10

def evaluate(strategy: Callable) -> float:
    opponent = (strategy, optimal_strategy)
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)

        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            logging.debug(f"status: After player {player} -> {nim}")
            player = 1 - player
        if player == 1:
            won += 1
    return won/NUM_MATCHES

In [264]:
def evaluate_evolve(strategy:Callable) -> float:
    opponent = (strategy, gabriele)
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)

        player = 0
        while nim:
            ply = opponent[player](nim)
            nim.nimming(ply)
            logging.debug(f"status: After player {player} -> {nim}")
            player = 1 - player
        if player == 1:
            won+=1
    return won/NUM_MATCHES

## Oversimplified match

In [265]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (make_strategy({'p': .1}), optimal_strategy)

nim = Nim(11)
logging.debug(f"status: Initial board -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 13 15 17 19 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 7 15 17 19 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 7 15 17 3 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 11 7 15 11 3 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 11 7 1 11 3 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 7 11 7 1 11 3 2>
DEBUG:root:status: After player 0 -> <1 3 5 7 7 3 7 1 11 3 2>
DEBUG:root:status: After player 1 -> <1 3 5 7 7 3 7 1 3 3 2>
DEBUG:root:status: After player 0 -> <1 3 5 6 7 3 7 1 3 3 2>
DEBUG:root:status: After player 1 -> <0 3 5 6 7 3 7 1 3 3 2>
DEBUG:root:status: After player 0 -> <0 3 5 6 6 3 7 1 3 3 2>
DEBUG:root:status: After player 1 -> <0 2 5 6 6 3 7 1 3 3 2>
DEBUG:root:status: After player 0 -> <0 2 5 6 6 3 7 0 3 3 2>
DEBUG:root:status: After player 1 -> <0 2 4 6 6 3 7 0 3 3 2>
DEBUG:root:status: After player 0 -> <0 2 4 6 6 3 2 0 3 3 2>